# Example for utilization of powerfactory-utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib

from powerfactory_utils.exporter import PowerfactoryExporter
from powerfactory_utils.exporter import export_powerfactory_data
from powerfactory_utils.schema.steadystate_case.case import Case as SteadyStateCase
from powerfactory_utils.schema.topology.topology import Topology
from powerfactory_utils.schema.topology_case.case import Case as TopologyCase

In [ ]:
PROJECT_NAME = "$PROJECT_NAME"  # may be also full path "dir_name\project_name"
GRID_NAME = "$GRID_NAME"
EXPORT_PATH = pathlib.Path("export")
PF_USER_PROFILE = "$USER_PROFILE"    # specification may be necessary

## Export using the export function running in a new process with default parameters

In [ ]:
# As the export function is executed in a process that is terminated after execution, the PowerFactory API is also closed.
export_powerfactory_data(export_path=EXPORT_PATH, project_name=PROJECT_NAME, grid_name=GRID_NAME, powerfactory_user_profile=PF_USER_PROFILE)

## Export using an exporter instance

In [ ]:
with PowerfactoryExporter(project_name=PROJECT_NAME, grid_name=GRID_NAME, powerfactory_user_profile=PF_USER_PROFILE) as exporter:
    # for this example switch to base scenario
    exporter.switch_scenario(scen="base")
    # exporter will generate now three JSON files: topology, topology_case, steadystate_case
    # topology_name is set by default to "GRID_NAME_topology.json"
    success = exporter.export(EXPORT_PATH, topology_name=f"{GRID_NAME}", topology_case_name=f"{GRID_NAME}", steadystate_case_name=f"{GRID_NAME}")
    assert success

### Check JSON Import and display exported JSON

In [ ]:
t = Topology.from_file(EXPORT_PATH / f"{GRID_NAME}_topology.json")
js = t.json()
t2 = Topology.from_json(js)
assert t2 == t  # will be thrown an error if the content is the same but the sequence not

In [ ]:
print(t2.json(indent=2))

In [ ]:
tc = TopologyCase.from_file(EXPORT_PATH / f"{GRID_NAME}_topology_case.json")
js2 = tc.json()
tc2 = TopologyCase.from_json(js2)
assert tc2 == tc  # will be thrown an error if the content is the same but the sequence not

In [ ]:
print(tc2.json(indent=2))

In [ ]:
ssc = SteadyStateCase.from_file(EXPORT_PATH / f"{GRID_NAME}_steadystate_case.json")
js2 = ssc.json()
ssc2 = SteadyStateCase.from_json(js2)
assert ssc2 == ssc  # will be thrown an error if the content is the same but the sequence not

In [ ]:
print(ssc2.json(indent=2))

### Redo export for different scenario

In [ ]:
with PowerfactoryExporter(project_name=PROJECT_NAME, grid_name=GRID_NAME, powerfactory_user_profile=PF_USER_PROFILE) as exporter:
    # switch to scenario "OutOfService_Set1" and export topology_case and steadystate_case
    success = exporter.export_scenario(export_path=EXPORT_PATH,scenario_name="outOfService_set1", topology_case_name=f"{GRID_NAME}_Set1", steadystate_case_name=f"{GRID_NAME}_Set1")
    assert success
    # finally, one can also export a specific steadystate_case
    data = exporter.compile_powerfactory_data()
    meta = exporter.create_meta_data(data=data)
    success = exporter.export_steadystate_case(meta=meta, data=data, filepath=EXPORT_PATH, steadystate_case_name=f"{GRID_NAME}_Set2")
    assert success

### Display raw JSON schema

In [ ]:
print(Topology.schema_json(indent=2))

In [ ]:
print(TopologyCase.schema_json(indent=2))

In [ ]:
print(SteadyStateCase.schema_json(indent=2))